In [1]:
import Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Generating Word Quiz

In [4]:
from main.quiz import *

temp = 0.7
dict_sheet_name = "Tua_List"
gsheet_name = "New Chinese Words"

In [5]:
word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
word_dict.head(2)

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Word Category,Word Rarity,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Added Date,Num_Quiz_Attempt,Num_Correct,Num_Wrong,Last_Quiz
1,1,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Support,Common,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,2024-09-21,0,0,0,
2,2,包裹,bao1 guo3,bao1 guo3,Noun,Object,Common,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2024-09-21,0,0,0,


In [7]:
quiz_df_show, answer_key = generate_pinyin_and_meaning_quiz(
    word_dict, 
    id_column='Word Id', 
    random_params={
        'num_words': 5, 
        'date_filter': '2024-10-13', 
        #'category_filter': 'Fashion'
        'rarity_filter': 'Common'
        }
    )

quiz_df_no_id = quiz_df_show.drop(columns=['Word Id'])
quiz_df_no_id

,Word,Sentence
0,表现,这项研究的表现令人满意。
1,医生,他是一名优秀的医生。
2,乘客,车上有很多乘客。
3,牵,他牵着狗散步。
4,继续,请继续努力工作。


In [8]:
pinyin_answer = 'biao3 xian4, yi1 sheng1, cheng4 ke4, qian1, ji4 xu1'.split(', ')
meaning_answer = 'represent, doctor, passengers, to lead, continue'.split(', ')

In [9]:
quiz_result = evaluate_pinyin_and_meaning_quiz(
    answer_key_df = answer_key, 
    quiz_df = quiz_df_show, 
    pinyin_answer = pinyin_answer, 
    meaning_answer = meaning_answer
)

update_quiz_score(
    quiz_result_df = quiz_result, 
    gsheet_name = gsheet_name, 
    wks_name = 'Tua_List'
    )

quiz_result

,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction,Right Score,Wrong Score,Last Quiz
Word Id,,,,,,,,,,,
493,表现,这项研究的表现令人满意。,biao3xian4,yes,,represent,no,"Means ""performance"" in this context.",0,1,2024-10-20
458,医生,他是一名优秀的医生。,yi1sheng1,yes,,doctor,yes,,1,0,2024-10-20
796,乘客,车上有很多乘客。,cheng4ke4,no,chéng kè,passengers,yes,,0,1,2024-10-20
385,牵,他牵着狗散步。,qian1,yes,,to lead,no,"Means ""to hold"" or ""to walk with"" in this cont...",0,1,2024-10-20
540,继续,请继续努力工作。,ji4xu1,no,jì xù,continue,yes,,0,1,2024-10-20


## Manual Code

In [ ]:
quiz_result_export = quiz_result.copy()
quiz_result_export['Right Score'] = np.where((quiz_result_export['Meaning Correct']=="yes")&(quiz_result_export['Pinyin Correct']=="yes"), 1, 0)
quiz_result_export['Wrong Score'] = np.where(quiz_result_export['Right Score']==0, 1, 0)
quiz_result_export['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index()

In [ ]:
word_dict_quiz_export = word_dict.merge(quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_export) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score'])